<center><h1>Bottlenecking VGG16 for Gender</h1></center>

This notebook takes a pre-trained VGG16 network on faces and bottlenecks it for gender classification

In [1]:
import numpy as np

from keras.models              import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers              import Flatten, Dense, Dropout, BatchNormalization,  Activation, MaxPooling2D, ZeroPadding2D, Convolution2D
from keras.utils.data_utils    import get_file

Using TensorFlow backend.


## 1. Instantiating the VGG16-Faces convolutional blocks

In [2]:
img_width, img_height  = 150, 150
TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/rcmalli/keras-vggface/releases/download/v1.0/rcmalli_vggface_tf_weights_tf_ordering_notop.h5'
weights_path           = get_file('rcmalli_vggface_tf_weights_tf_ordering_notop.h5',
                         TF_WEIGHTS_PATH_NO_TOP, cache_subdir='models')

In [3]:
# build the VGG16 network
vgg_model = Sequential()
vgg_model.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))

vgg_model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="tf"))

vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="tf"))

vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="tf"))

vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="tf"))

vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
vgg_model.add(ZeroPadding2D((1, 1)))
vgg_model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
vgg_model.add(MaxPooling2D((2, 2), strides=(2, 2), dim_ordering="tf"))

In [4]:
# Loading the VGG16-Faces weights
vgg_model.load_weights(weights_path)

## 2. Prepping the training data

In [5]:
train_data_dir = "../data/face_image_project/keras_format/train"

datagen   = ImageDataGenerator()
generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(150, 150),
            batch_size=3000,
            class_mode='categorical',  
            shuffle=True) 

X_train, Y_train = generator.next()

Found 14859 images belonging to 3 classes.


In [6]:
bottleneck_features_train = vgg_model.predict(X_train, 40)

In [7]:
bottleneck_weights_path = "vgg16_faces_gender_bottleneck.npy"
np.save(open(bottleneck_weights_path, 'w'), bottleneck_features_train)
train_data = np.load(open(bottleneck_weights_path))

In [8]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Flatten(input_shape=train_data.shape[1:]))
vainilla_cnn.add(Dense(output_dim=200))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=100, input_dim=200))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dropout(0.5))
vainilla_cnn.add(Dense(output_dim=3))
vainilla_cnn.add(Activation("softmax"))

In [9]:
vainilla_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
vainilla_cnn.fit(train_data, Y_train,batch_size=40, nb_epoch=2000)

Epoch 1/2000
3000/3000 [==============================] - 2s - loss: 0.9993 - acc: 0.5377      

In [ ]:
vainilla_cnn.save('saved')
vainilla_cnn.save_weights('my_model_weights.h5')

## VGG-Face
Source: https://github.com/rcmalli/keras-vggface

In [ ]:
nb_class = 3
hidden_dim = 512
image_input = Input(shape=(200, 200, 3))
vgg_model = VGGFace(input_tensor=image_input, include_top=False)
last_layer = vgg_model.get_layer('pool5').output

x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(image_input, out)

for layer in custom_vgg_model.layers[:25]:
    layer.trainable = False

custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


img_height = 200
img_width  = 200
nb_epoch = 100
nb_train_samples = 32
nb_validation_samples = 32
train_data_dir = "../data/face_image_project/keras_format/train"
validation_data_dir ="../data/face_image_project/keras_format/test"

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

# fine-tune the model
history = custom_vgg_model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

In [ ]:
nb_class = 3
hidden_dim = 512
image_input = Input(shape=(200, 200, 3))
vgg_model = VGGFace(input_tensor=image_input, include_top=False)
last_layer = vgg_model.get_layer('pool5').output

x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(image_input, out)

for layer in custom_vgg_model.layers[:20]:
    layer.trainable = False

custom_vgg_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


img_height = 200
img_width  = 200
nb_epoch = 100
nb_train_samples = 32
nb_validation_samples = 32
train_data_dir = "../data/face_image_project/keras_format/train"
validation_data_dir ="../data/face_image_project/keras_format/test"

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')

# fine-tune the model
history = custom_vgg_model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)